In [ ]:
import torch
from torch import nn

import torchvision
from torchvision import datasets

from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

print(f'PyTorch version: {torch.__version__}\n torchvision version: {torchvision.__version__}')

In [ ]:
train_data = datasets.FashionMNIST(
    root='data',
    train = True,
    download = True,
    transform = ToTensor(),
    target_transform = None
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

In [ ]:
image, label = train_data[0]
image, label

In [ ]:
image.shape

In [ ]:
len(train_data.data), len(train_data.targets), len(test_data.data), len(test_data.targets)

In [ ]:
class_names = train_data.classes
class_names

In [ ]:
train_data.targets

In [ ]:
image.shape

Visualizing

In [ ]:
import matplotlib.pyplot as plt
image, label = train_data[0]
print(image.shape)
plt.imshow(image.squeeze())
plt.title(label)

In [ ]:
plt.imshow(image.squeeze(), cmap ='gray')
plt.title(class_names[label])

In [ ]:
torch.manual_seed(42)
fig = plt.figure(figsize= (9, 9))
rows, cols = 4,4
for i in range(1, rows* cols + 1):
  random_idx = torch.randint(0, len(train_data), size = [1]).item()
  img, label = train_data[random_idx]
  fig.add_subplot(rows, cols, i)
  plt.imshow(img.squeeze(), cmap = 'gray')
  plt.title(class_names[label])
  plt.axis(False)

In [ ]:
train_data, test_data

### Prepare Dataloader

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32

train_dataloader = DataLoader(dataset = train_data, batch_size = BATCH_SIZE, shuffle = True)
test_dataloader = DataLoader(dataset = test_data, batch_size = BATCH_SIZE, shuffle = False)

train_dataloader, test_dataloader

In [ ]:
train_dataloader.dataset

In [ ]:
len(train_dataloader)

In [ ]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

In [ ]:
#torch.manual_seed(42)
random_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, label = train_features_batch[random_idx], train_labels_batch[random_idx]
plt.imshow(img.squeeze(), cmap = 'gray')
plt.title(class_names[label])
plt.axis(False)

### Building Model

Basline Model

In [ ]:
flatten_model = nn.Flatten()

x= train_features_batch[0]

output = flatten_model(x)

print(x.shape)
print(output.shape)


In [ ]:
from torch import nn
class FashionMNISTModelV0(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = input_shape, out_features = hidden_units),
        nn.Linear(in_features = hidden_units, out_features = output_shape)
    )
  def forward(self, x):
    return self.layer_stack(x)

In [ ]:
torch.manual_seed(42)
model_0 = FashionMNISTModelV0(
    input_shape = 784,
    hidden_units = 10,
    output_shape = len(class_names)
)
model_0.to('cpu')

Loss , optimizer and Evaluation Metrics

In [ ]:
import requests
from pathlib import Path

if Path('helper_functions.py').is_file():
  print('helper_functions.py already exists')
else:
  print('Downloading helper_functions.py')
  request = requests.get('https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py')

  with open('helper_functions.py', 'wb') as f:
    f.write(request.content)

In [ ]:
from helper_functions import accuracy_fn

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model_0.parameters(), lr = 0.1)

Timer function

In [ ]:
from timeit import default_timer as timer
def print_train_time(start:float, end: float, device: torch.device = None):
  total_time = end - start
  print(f'Train time on {device}: {total_time:.3f} seconds')
  return total_time

Training Loop

In [ ]:
from tqdm.auto import tqdm

torch.manual_seed(42)
train_time_start_on_cpu = timer()

epochs = 10

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n----")
  train_loss = 0
  for batch, (X, Y) in enumerate(train_dataloader):
    model_0.train()

    y_preds = model_0(X)

    loss = loss_fn(y_preds, Y)
    train_loss += loss

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    if batch % 400 == 0:
      print(f"Looked at {batch * len(X)/len(train_dataloader.dataset)} samples. ")

  train_loss /=len(train_dataloader)

  test_loss, test_acc =0, 0
  model_0.eval()
  with torch.inference_mode():
    for x, y in test_dataloader:
      test_preds = model_0(x)

      test_loss += loss_fn(test_preds, y)

      test_acc += accuracy_fn(y_true= y, y_pred = test_preds.argmax(dim=1))

    test_loss /= len(test_dataloader)

    test_acc /= len(test_dataloader)

  print(f'\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n')

train_time_end_on_cpu = timer()
total_train_time_model_0 = print_train_time(start= train_time_start_on_cpu, end = train_time_end_on_cpu, device = str(next(model_0.parameters()).device) )







### 4.Make predictions

In [ ]:
torch.manual_seed(42)
def eval_model(model: torch.nn.Module, data_loader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, accuracy_fn):
  loss, acc = 0, 0
  model.eval()

  with torch.inference_mode():
    for x, y in data_loader:
      y_preds = model(x)

      loss += loss_fn(y_preds, y)
      acc += accuracy_fn(y_true = y, y_pred = y_preds.argmax(dim= 1))

    loss /= len(data_loader)
    acc /= len(data_loader)

  return {
      "model_name": model.__class__.__name__,
      "model_loss": loss.item(),
      "model_acc": acc
  }

model_0_results = eval_model(model= model_0, data_loader= test_dataloader, loss_fn = loss_fn, accuracy_fn = accuracy_fn)
model_0_results

### Setup device Agnostic code

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

### Model 1

In [ ]:
class FashionMNISTModelV1(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = input_shape, out_features = hidden_units),
        nn.ReLU(),
        nn.Linear(in_features = hidden_units, out_features = output_shape),
        nn.ReLU()
    )

  def forward(self, x: torch.Tensor):
    return self.layer_stack(x)



In [ ]:
torch.manual_seed(42)
model_1 = FashionMNISTModelV1(input_shape= 784, hidden_units = 10, output_shape = len(class_names)).to(device)

next(model_1.parameters()).device

### Setup loss, optimizer and evalution metrics

In [ ]:
from helper_functions import accuracy_fn
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model_1.parameters(), lr = 0.1)

###Functioning Training and Test Loops

In [ ]:
def train_step(model: torch.nn.Module, data_loader : torch.utils.data.DataLoader, loss_fn : torch.nn.Module, optimizer: torch.optim.Optimizer, accuracy_fn: accuracy_fn, device: torch.device = device):
  train_loss, train_acc = 0, 0
  model.to(device)
  for batch, (X, Y) in enumerate(data_loader):
    x = X.to(device)
    y = Y.to(device)

    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    train_loss += loss
    train_acc += accuracy_fn(y_true = y, y_pred = y_pred.argmax(dim=1))

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

  train_loss /= len(data_loader)
  train_acc /= len(data_loader)


  print(f"Train loss: {train_loss:.5f} | Train accuarcy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader, model: torch.nn.Module, loss_fn: torch.nn.Module, accuracy_fn, device: torch.device = device):
  test_loss, test_acc = 0, 0
  model.to(device)

  model.eval()

  with torch.inference_mode():
    for X, Y in data_loader:
      x, y = X.to(device), Y.to(device)

      test_preds = model(x)

      test_loss += loss_fn(test_preds, y)

      test_acc += accuracy_fn(y_true= y, y_pred = test_preds.argmax(dim =1))

  test_loss /= len(data_loader)
  test_acc /= len(data_loader)


  print(f"Test loss: {test_loss:.5f} |  Test accuracy: {test_acc:.2f}%\n")

In [ ]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    model.to(device)
    for batch, (X, y) in enumerate(data_loader):
        # Send data to GPU
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.to(device)
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode():
        for X, y in data_loader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )

        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
torch.manual_seed(42)

# Measure time
from timeit import default_timer as timer
train_time_start_on_gpu = timer()

epochs = 3
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=train_dataloader,
        model=model_1,
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn
    )
    test_step(data_loader=test_dataloader,
        model=model_1,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn
    )

train_time_end_on_gpu = timer()
total_train_time_model_1 = print_train_time(start=train_time_start_on_gpu,
                                            end=train_time_end_on_gpu,
                                            device=device)

In [ ]:
torch.manual_seed(42)
model_1_results = eval_model(model = model_1, data_loader = test_dataloader, loss_fn = loss_fn, accuracy_fn = accuracy_fn)
model_1_results

In [ ]:
torch.manual_seed(42)
def eval_model(model: torch.nn.Module, data_loader: torch.utils.data.DataLoader, loss_fn : torch.nn.Module, accuracy_fn , device: torch.device = device):
  loss, acc = 0, 0
  model.eval()
  with torch.inference_mode():
    for X, Y in data_loader:
      x, y = X.to(device), Y.to(device)

      y_preds = model(x)

      loss += loss_fn(y_preds, y)

      acc += accuracy_fn(y_true = y, y_pred = y_preds.argmax(dim=1))

    loss /= len(data_loader)
    acc /= len(data_loader)
  return {
      "model_name": model.__class__.__name__,
      "model_loss": loss.item(),
      "model_acc": acc
  }

model_1_results = eval_model(model= model_1, data_loader = test_dataloader, loss_fn = loss_fn, accuracy_fn = accuracy_fn, device = device)
model_1_results



In [ ]:
model_0_results

### Model 2: Building CNN(convolution neural network)

In [ ]:
class FashionMNISTModelV2(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    super().__init__()
    self.block_1 = nn.Sequential(
        nn.Conv2d(in_channels = input_shape, out_channels= hidden_units, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.Conv2d(in_channels= hidden_units, out_channels = hidden_units, kernel_size = 3, stride = 1, padding =1),
        nn.ReLU(),
        nn.Conv2d(in_channels= hidden_units, out_channels = hidden_units, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride =2)
    )
    self.block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, 3, padding= 1),
        nn.ReLU(),
        nn.MaxPool2d(2)
   )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_units * 7 *7, out_features = output_shape)
    )
  def forward(self, x: torch.Tensor):
    x = self.block_1(x)
    x = self.block_2(x)
    x = self.classifier(x)

    return x

torch.manual_seed(42)
model_2 = FashionMNISTModelV2(input_shape=1, hidden_units = 10, output_shape = len(class_names)).to(device)
model_2

In [ ]:
torch.manual_seed(42)

images = torch.randn(size = (32, 3, 64, 64))
test_image = images[0]
print(images.shape)
print(test_image.shape)
print(test_image)

In [ ]:
torch.manual_seed(42)

conv_layer = nn.Conv2d(in_channels=  3, out_channels= 10, kernel_size = 3, stride=1, padding=0)
conv_layer(test_image)

In [ ]:
test_image.shape

In [ ]:
test_image.unsqueeze(dim=0).shape

In [ ]:
conv_layer(test_image.unsqueeze(dim=0)).shape

In [ ]:
torch.manual_seed(42)
conv_layer_2 = nn.Conv2d(in_channels=3, out_channels= 10, kernel_size = (5,5), stride=2, padding=0)
conv_layer_2(test_image.unsqueeze(dim=0)).shape

In [ ]:
conv_layer_2.state_dict()

In [ ]:
print(f'conv_layer_2 weight shape: {conv_layer_2.weight.shape}')
print(conv_layer_2.bias.shape)

In [ ]:
max_pool_layer = nn.MaxPool2d(kernel_size = 2)
test_image_through_conv = conv_layer(test_image.unsqueeze(dim=0))
print(test_image_through_conv.shape)
test_image_through_conv_and_max_pool = max_pool_layer(test_image_through_conv)
print(test_image_through_conv_and_max_pool.shape)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params= model_2.parameters(), lr=0.1)

In [ ]:
torch.manual_seed(42)

from timeit import default_timer as timer

train_time_start_model_2 = timer()

epochs = 3

for epoch in tqdm(range(epochs)):
  print(epoch)

  train_step(data_loader = train_dataloader, model = model_2, loss_fn = loss_fn, optimizer = optimizer, accuracy_fn = accuracy_fn, device = device)
  test_step(data_loader = test_dataloader, model= model_2, loss_fn = loss_fn, accuracy_fn = accuracy_fn, device = device)

train_time_end_model_2 = timer()
total_train_time_model_2 = print_train_time(start = train_time_start_model_2, end = train_time_end_model_2, device = device)


In [ ]:
model_2_results = eval_model(model= model_2, data_loader = test_dataloader, loss_fn = loss_fn, accuracy_fn = accuracy_fn)

model_2_results

### Compare model results

In [ ]:
import pandas as pd
compare_results = pd.DataFrame([model_0_results, model_1_results, model_2_results])

compare_results

In [ ]:
compare_results.set_index("model_name")['model_acc'].plot(kind="barh")
plt.xlabel("accurcy %")
plt.ylabel("model");

##Make and Evaluate random predictions

In [ ]:
def make_predictions(model: torch.nn.Module, data: list, device: torch.device = device):
  pred_probs = []
  model.eval()
  with torch.inference_mode():
    for sample in data:
      sample = torch.unsqueeze(sample, dim= 0).to(device)
      pred_logit = model(sample)
      pred_prob = torch.softmax(pred_logit.squeeze(), dim =0)
      pred_probs.append(pred_prob.cpu())

  return torch.stack(pred_probs)

In [ ]:
import random

random.seed(42)
test_samples = []

test_labels = []
for sample, label in random.sample(list(test_data), k=9):
  test_samples.append(sample)
  test_labels.append(label)

print(test_samples[0].shape)

In [ ]:
pred_probs = make_predictions(model= model_2, data = test_samples)
pred_probs[:2]

In [ ]:

pred_classes = pred_probs.argmax(dim=1)
pred_classes

In [ ]:
test_labels, pred_classes

In [ ]:
plt.figure(figsize=(9, 9))
nrows = 3
ncols = 3
for i, sample in enumerate(test_samples):
  plt.subplot(nrows, ncols, i+1)
  plt.imshow(sample.squeeze(), cmap="gray")

  pred_label = class_names[pred_classes[i]]

  truth_label = class_names[test_labels[i]]
  plt.title(f"Pred: {pred_label} | Truth: {truth_label}")
  plt.axis("off")


### Confusion Matrix

In [ ]:
from tqdm.auto import tqdm

y_preds = []

model_2.eval()
with torch.inference_mode():
  for X, Y in tqdm(test_dataloader, desc="Making predictions"):
    x, y = X.to(device), Y.to(device)

    y_logit = model_2(x)
    y_pred = torch.softmax(y_logit, dim=1).argmax(dim=1)

    y_preds.append(y_pred.cpu())

y_pred_tensor = torch.cat(y_preds)

In [ ]:
try:
  import trochmetrics, mlxtend
  print(f"mlxtend version: {mlxtend.__version__}")
except:
  !pip install -q torchmetrics -U mlxtend
  import torchmetrics, mlxtend
  print(f"mlxtend version: {mlxtend.__version__}")

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

confmat = ConfusionMatrix(num_classes = len(class_names), task = "multiclass" )
confmat_tensor = confmat(preds= y_pred_tensor, target= test_data.targets)

fig, ax = plot_confusion_matrix(
    conf_mat = confmat_tensor.numpy(),
    class_names = class_names,
    figsize = (10, 7)

)

###Save and Load Best Model

In [ ]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents= True, exist_ok = True)

MODEL_NAME = "pytorch_computer_vision_model_2_fashionMNIST.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj= model_2.state_dict(), f = MODEL_SAVE_PATH)

In [ ]:
loaded_model_2 = FashionMNISTModelV2(input_shape=1, hidden_units = 10, output_shape= 10)

loaded_model_2.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

loaded_model_2 = loaded_model_2.to(device)

In [ ]:
torch.manual_seed(42)

loaded_model_2_results = eval_model(
    model = loaded_model_2,
    data_loader = test_dataloader,
    loss_fn = loss_fn,
    accuracy_fn = accuracy_fn
)
loaded_model_2_results

In [ ]:
model_2_results

In [ ]:
torch.isclose(torch.tensor(model_2_results["model_loss"]),
              torch.tensor(loaded_model_2_results["model_loss"]),
              atol = 1e-08,
              rtol = 0.0001)